In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve
from urllib import request

In [2]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")
X_test.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,1,101,50,15,36,24.2,0.526,26
1,4,156,75,0,0,48.3,0.238,32
2,1,97,66,15,140,23.2,0.487,22


In [23]:
def get_prediction(x):

    Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age = x

    body = {'Pregnancies': Pregnancies,
            'Glucose': Glucose,
            'BloodPressure': BloodPressure,
            'SkinThickness': SkinThickness,
            'Insulin': Insulin,
            'BMI': BMI,
            'DiabetesPedigreeFunction': DiabetesPedigreeFunction,
            'Age': Age}

    myurl = "http://127.0.0.1:5000/predict"
    req = request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')
    req.add_header('Content-Length', len(jsondataasbytes))
    response = request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [24]:
%%time
predictions = X_test.apply(lambda x: get_prediction(x), 1)

CPU times: total: 125 ms
Wall time: 1.8 s


In [8]:
predictions.head()

0    0.01
1    0.79
2    0.03
3    0.67
4    0.36
dtype: float64

In [26]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

fscore = (2 * precision * recall) / (precision + recall)

ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]:.3f}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}\n')

Best Threshold=0.400, F-Score=0.718, Precision=0.681, Recall=0.758



In [7]:
roc_auc_score(y_score=predictions.values, y_true=y_test)

0.7864656381486675